### RAG TEST

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from dotenv import load_dotenv 
import os 

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "plant-recommend" 
index = pc.Index(index_name) 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


query = """{
  "purpose": "거실에 포인트가 될 식물",
  "preferred_style": "미니멀",
  "preferred_color": "#A1B07C",
  "plant_type": "관엽식물",
  "season": "봄",
  "humidity": "보통",
  "has_dog": false,
  "has_cat": false,
  "isAirCond": true,
  "watering_frequency": "주 1회",
  "user_experience": "초보자",
  "emotion": "깔끔하고 차분한 분위기"
}
"""

query_embeddings = embeddings.embed_query(query)        
query_response = \
response = index.query(
    top_k = 3,                 # 결과 반환 갯수
    include_values=False,      # 벡터 임베딩 반환 여부
    include_metadata=True,     # 메타 데이터 반환 여부
    vector=query_embeddings    # 검색할 벡터 임베딩
)
print(response)

c:\Users\Playdata\AppData\Local\anaconda3\envs\project3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'matches': [{'id': '343',
              'metadata': {'dataNo': '343',
                           'difficulty': '쉬움',
                           'fContent': '꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 '
                                       '겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 '
                                       '청소년들 선물용으로도 좋은 아이템이다.',
                           'fDay': '8',
                           'fGrow': '5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 '
                                    '잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 '
                                    '잘 증식된다.',
                           'fMonth': '12',
                           'fType': '돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 '
                                    "있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.",
                           'fUse': '다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 '
                                   '웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다.',
                           'flowLang': '인기, 설레임',
                         

In [2]:
print(response['matches'][0].metadata)
print(response['matches'][0].score)

print(response['matches'][1].metadata)
print(response['matches'][1].score)

print(response['matches'][2].metadata)
print(response['matches'][2].score)

{'dataNo': '343', 'difficulty': '쉬움', 'fContent': '꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다.', 'fDay': '8', 'fGrow': '5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다.', 'fMonth': '12', 'fType': "돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.", 'fUse': '다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다.', 'flowLang': '인기, 설레임', 'flowNm': '칼랑코에', 'imgUrl1': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-1.jpg', 'imgUrl2': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-2.jpg', 'imgUrl3': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-3.jpg', 'interior': '칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다.', 'isAirCond': True, 'isToxicToCat': False, 'isToxicToDog': False, 'style': '모던, 미니멀리즘, 보헤미안', 'text': "인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은

In [7]:
# LLM 평가 함수
from langchain_openai import ChatOpenAI

def test_llm(query, response):
    prompt = f"""
        너는 RAG 평가자야.
        query문과 RAG를 통해 얻은 response의 유사도를 비교해서 점수로 내.
        
        ### 비교 대상 ###
        query = {query}
        response = {response}
    """
        
    model = ChatOpenAI(
        model='gpt-4o',
        temperature=1
    )
    result = model.invoke([prompt])
    
    return result

In [ ]:
res = test_llm(query, response['matches'][0].metadata)
print(res)

content='### RAG 평가 ###\n\n1. **목적 및 스타일 적합성**:\n   - Query에서 사용자는 "거실에 포인트가 될 식물"을 원하고, 스타일로는 "미니멀"을 선호한다고 했습니다. \n   - Response에 설명된 칼랑코에는 다양한 꽃 색상이 있어 시각적인 포인트가 될 수 있으며, "모던한 스타일의 화분"으로 미니멀리즘 스타일에 적합하다고 제안하고 있습니다.\n\n2. **색상 적합성**:\n   - Query에서는 선호하는 색상이 "#A1B07C"로 주어졌으나, Response에서는 직접적으로 이 색상에 대한 언급이 없습니다. 그러나 다양한 꽃 색상이 있다는 점에서 어느 정도 유연하게 해석될 수 있습니다.\n\n3. **식물 유형 및 계절 적합성**:\n   - Query에서 관엽식물을 원하고, 봄을 언급했으며, Response에서 칼랑코에는 관화식물로 분류되어 있습니다. 이는 완벽한 일치가 아니지만 유사성을 가지고 있습니다. 또한, 5도 이상의 조건이면 겨울철에도 잘 자란다고 설명되어 있으나, 봄에 대한 직접적인 언급은 없습니다.\n\n4. **습도 및 물 주기 적합성**:\n   - Query에서 보통의 습도와 주 1회의 물 주기를 언급했고, Response에서 \'다육성 식물\'이고 \'물도 좋아하며 주 1회 물 주기\'가 적절하다고 설명되어 있어 요구 조건과 잘 맞습니다.\n\n5. **반려동물 유해성 및 주변 환경 조건**:\n   - Query에서 반려동물이 없음을 언급하였고, Response에 따르면 칼랑코에는 강아지와 고양이 모두에게 안전하다고 되어 있어 문제가 없습니다. 또한, 에어컨이 있는 것을 고려하여 찬 기후에도 잘 견딘다는보충 정보가 있습니다.\n\n6. **사용자 경험 및 정서적 연관성**:\n   - Query에서는 초보자를 위한 식물과 "깔끔하고 차분한 분위기"를 희망했으며, Response에서는 \'쉬움\' 난이도와 함께 창가에 두어 공간에 생기를 부여한다고 설명하여 이를 지원합니다. \n\n**최종 

In [9]:
print(res.content)


### RAG 평가 ###

1. **목적 및 스타일 적합성**:
   - Query에서 사용자는 "거실에 포인트가 될 식물"을 원하고, 스타일로는 "미니멀"을 선호한다고 했습니다. 
   - Response에 설명된 칼랑코에는 다양한 꽃 색상이 있어 시각적인 포인트가 될 수 있으며, "모던한 스타일의 화분"으로 미니멀리즘 스타일에 적합하다고 제안하고 있습니다.

2. **색상 적합성**:
   - Query에서는 선호하는 색상이 "#A1B07C"로 주어졌으나, Response에서는 직접적으로 이 색상에 대한 언급이 없습니다. 그러나 다양한 꽃 색상이 있다는 점에서 어느 정도 유연하게 해석될 수 있습니다.

3. **식물 유형 및 계절 적합성**:
   - Query에서 관엽식물을 원하고, 봄을 언급했으며, Response에서 칼랑코에는 관화식물로 분류되어 있습니다. 이는 완벽한 일치가 아니지만 유사성을 가지고 있습니다. 또한, 5도 이상의 조건이면 겨울철에도 잘 자란다고 설명되어 있으나, 봄에 대한 직접적인 언급은 없습니다.

4. **습도 및 물 주기 적합성**:
   - Query에서 보통의 습도와 주 1회의 물 주기를 언급했고, Response에서 '다육성 식물'이고 '물도 좋아하며 주 1회 물 주기'가 적절하다고 설명되어 있어 요구 조건과 잘 맞습니다.

5. **반려동물 유해성 및 주변 환경 조건**:
   - Query에서 반려동물이 없음을 언급하였고, Response에 따르면 칼랑코에는 강아지와 고양이 모두에게 안전하다고 되어 있어 문제가 없습니다. 또한, 에어컨이 있는 것을 고려하여 찬 기후에도 잘 견딘다는보충 정보가 있습니다.

6. **사용자 경험 및 정서적 연관성**:
   - Query에서는 초보자를 위한 식물과 "깔끔하고 차분한 분위기"를 희망했으며, Response에서는 '쉬움' 난이도와 함께 창가에 두어 공간에 생기를 부여한다고 설명하여 이를 지원합니다. 

**최종 점수: 8/10**

- 이유: 대부분의 요소에서 query의 조